compare results from 8A and irf cca results (6A)

07/22/2020

modified 08/16/2020 - for only selected tissues for mpra

In [97]:
import os, glob
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib as plt

In [70]:
sel_tissues = ['Colon','GDSD6','GM12878','Melanocytes']

In [75]:
save_dir = '../data/processed/fig4_modelling/tf_tf_pairs_stability/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [76]:
# tf tf enrichment results
enrichment_path =  '../data/processed/fig4_modelling/tf_tf_pairs/'
sorted(glob.glob(os.path.join(enrichment_path,'*info.csv')))

['../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Bladder_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Bladder_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_pro_pro_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_loop_loop_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/ex

In [77]:
# stability_path = '../data/processed/fig4_modelling/irf_manual/'

stability_path = '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/'
sorted(glob.glob(os.path.join(stability_path, '*boosted_stability_score.csv')))

['../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Colon_Colon_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_GDSD6_GDSD6_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_GM12878_GM12878_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Melanocytes_Melanocytes_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/test1_GDSD6_GDSD6_boosted_stability_score.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/test_GDSD6_GDSD6_boosted_stability_score.csv']

In [78]:
group_dict={'Airway':'purple','Astrocytes':'blue','Bladder':'purple','Colon':'green','Esophageal':'green','GDSD6':'purple','GM12878':'grey','HMEC':'purple','Melanocytes':'blue','Ovarian':'green',
'Pancreas':'green','Prostate':'purple','Renal':'green','Thyroid':'green','Uterine':'purple'}
    


In [79]:
def get_vocab_pair(vocab):
    vocab_arr = vocab.split('::')
    vocab_arr_trim = [x.split('_')[0] for x in vocab_arr]
    return '::'.join(vocab_arr_trim)


In [122]:
def merge_methods(tissue, save=None,loop_type='pro_pro', expr=True,group=False):
    print('++++++++',tissue, 'group', group, 'expr',expr, 'loop_type', loop_type)
    if expr:
        expr_file = os.path.join(enrichment_path, 'expr_'+tissue+'_' + loop_type + '_vocab_info.csv')
    else:
        expr_file = os.path.join(enrichment_path, 'unique_alltissue_'+loop_type+'_vocab_info.csv')
    if not os.path.isfile(expr_file):
        print('expr file not found, ', expr_file)
        return None
    else:
        print('loaded expr file: ', expr_file)
    df_enrich = pd.read_csv(expr_file, index_col=0)
    # df_enrich = pd.read_csv('../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.csv', index_col=0)
    df_enrich = df_enrich[df_enrich.tissue==tissue]
    df_enrich_vocab = df_enrich.vocab.unique()
    print('num vocab enrichment', df_enrich_vocab.shape)
    
    if group:
        stability_file = os.path.join(stability_path, 'expr_'+tissue+'_'+group_dict[tissue]+'_boosted_stability_score.csv')
    else:
        stability_file = os.path.join(stability_path, 'expr_'+tissue+'_'+tissue+'_boosted_stability_score.csv')
    if not os.path.isfile(stability_file):
        print('stability file not exist', stability_file)
        return None 
    else:
        print('loaded stability file', stability_file)
    df_stability = pd.read_csv(stability_file)
    df_stability['vocab'] = df_stability['index'].apply(get_vocab_pair)
    df_stability['vocab_num'] = df_stability['index'].apply(lambda x: len(x.split('::')))
    
    ##### MERGING ENRICH AND STABLE
#     df_enrich_stable =  df_enrich.merge(df_stability, how='inner',on='vocab',suffixes=('_tf','_model'))
    vocab_arr = [x.split('::') for x in df_enrich.vocab.unique()]
    e_vocab_stable = defaultdict(dict)
    for idx, row in df_stability.iterrows():
        s_vocab = row['vocab']
        s_vocab_arr = s_vocab.split('::')
        for e_vocab_arr in vocab_arr:
            enrich_in_stable = True
            for e_vocab in e_vocab_arr:
                if e_vocab not in s_vocab_arr:
                    enrich_in_stable = False

            if enrich_in_stable:
                e_vocab_stable['::'.join(e_vocab_arr)][s_vocab] = row['score']

        vocab_overlap_df = pd.DataFrame.from_dict(e_vocab_stable,orient='index').reset_index().melt(id_vars='index')
    vocab_overlap_df.columns = ['enrich_vocab','stability_vocab', 'stability_score']
    vocab_overlap_df= vocab_overlap_df[~vocab_overlap_df.stability_score.isna()].reset_index(drop=True)
    df_enrich_stable = df_enrich.merge(vocab_overlap_df, how='inner',left_on='vocab',right_on='enrich_vocab')

            
    print('df_enrich_stable', df_enrich_stable.shape)
    if df_enrich_stable.shape[0]==0:
        return
    if save is not None:
        print('saving',tissue)
        if expr:
            if group:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_group.csv')            
            else:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab.csv')
        else:
            if group:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_group_unique.csv')            
            else:
                df_enrich_stable.to_csv(save+ tissue+'_'+loop_type+'_vocab_unique.csv')


In [123]:
# testing
# df_enrich, df_stability = merge_methods('Colon',save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')
# df_enrich_stable = merge_methods('Colon',save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')

In [124]:
for tissue in sel_tissues:
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')
#     merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'pro_pro')# unique to not tissue
#     merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'pro_pro') #unique to only tissue
    merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'loop_loop')
#     merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'loop_loop')# unique to not tissue
#     merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'loop_loop') #unique to only tissue
        

++++++++ Colon group False expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_pro_pro_vocab_info.csv
num vocab enrichment (119,)
loaded stability file ../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Colon_Colon_boosted_stability_score.csv
df_enrich_stable (128, 17)
saving Colon
++++++++ Colon group False expr True loop_type loop_loop
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_loop_loop_vocab_info.csv
num vocab enrichment (220,)
loaded stability file ../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_Colon_Colon_boosted_stability_score.csv
df_enrich_stable (2037, 17)
saving Colon
++++++++ GDSD6 group False expr True loop_type pro_pro
loaded expr file:  ../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_pro_pro_vocab_info.csv
num vocab enrichment (1722,)
loaded stability file ../data/processed/fig4_modelling/tf_tf_pairs_stability_sherlock/expr_GDSD6_GDSD6_booste

In [125]:
# for tissue in ['Melanocytes']:
#     merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'pro_pro')
#     merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'pro_pro')# unique to not tissue
#     merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'pro_pro') #unique to only tissue
#     merge_methods(tissue,save=save_dir+'/', expr=True,group=False, loop_type= 'loop_loop')
#     merge_methods(tissue,save=save_dir+'/',expr=True, group=True,loop_type= 'loop_loop')# unique to not tissue
#     merge_methods(tissue,save=save_dir+'/',expr=False,group=False,loop_type= 'loop_loop') #unique to only tissue
        

In [ ]:

# df_stability = pd.read_csv('../data/processed/fig4_modelling/irf_manual/test_GDSD6_GDSD6_boosted_stability_score.csv')
# df_stability['vocab'] = df_stability['index'].apply(get_vocab_pair)

# df_enrich.merge(df_stability, how='inner',on='vocab',suffixes=('_tf','_model'))

In [126]:
# for tissue in normal_tissues:
    